In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from sklearn.model_selection import train_test_split
import classifier as cf
import seaborn as sns
sns.set_style("darkgrid")

In [2]:
traindata = pd.read_csv("data/processed_training_data.csv", nrows=10000)

In [3]:
def give_score(row):
    if row["booking_bool"] > 0:
        return int(5)
    if row["click_bool"] > 0:
        return int(1)
    return int(0)

In [4]:
traindata["score"] = traindata.apply(give_score, axis=1)
print(len(traindata))

10000


In [5]:
testdata = pd.read_csv("data/processed_testing_data.csv", nrows=5000)
print(len(testdata))

5000


In [6]:
print(traindata.columns)

Index(['Unnamed: 0', 'srch_id', 'date_time', 'site_id',
       'visitor_location_country_id', 'prop_country_id', 'prop_id',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_log_historical_price', 'position',
       'price_usd', 'promotion_flag', 'srch_destination_id',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool',
       'random_bool', 'click_bool', 'booking_bool', 'score'],
      dtype='object')


In [7]:
print(testdata.columns)

Index(['Unnamed: 0', 'srch_id', 'date_time', 'site_id',
       'visitor_location_country_id', 'prop_country_id', 'prop_id',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_log_historical_price', 'price_usd',
       'promotion_flag', 'srch_destination_id', 'srch_length_of_stay',
       'srch_booking_window', 'srch_adults_count', 'srch_children_count',
       'srch_room_count', 'srch_saturday_night_bool', 'random_bool', 'score'],
      dtype='object')


In [8]:
attrs = ["prop_starrating","prop_review_score" ,"prop_location_score1", 
         "prop_log_historical_price", "price_usd", "srch_length_of_stay"]
flag = "True"
y = traindata["score"]
X = traindata[attrs]
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.5, random_state=101)

In [9]:
gnb = cf.gaussian_naive_bayes(Xtrain, Xtest, ytrain, ytest, flag)

Gaussian Naive Bayes: cross vali score 0.9394070717424012 , accuracy score 0.9392


In [10]:
dt = cf.decision_tree(Xtrain, Xtest, ytrain, ytest, flag)

Decision Tree: cross vali score 0.9055990121858386 , accuracy score 0.8986


In [11]:
knn = cf.k_nearest_neighbours(Xtrain, Xtest, ytrain, ytest, flag)

K Nearest Neighbours: cross vali score 0.9546007084929241 , accuracy score 0.9538


In [12]:
sgd = cf.stochastic_gradient_descent(Xtrain, Xtest, ytrain, ytest, flag)

Stochastic Gradient Descent: cross vali score 0.9532011052969137 , accuracy score 0.9538


In [13]:
lr = cf.logistic_regression(Xtrain, Xtest, ytrain, ytest, flag)

C:\Users\Helia\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Helia\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

Logistic Regression: cross vali score 0.9548003092913273 , accuracy score 0.9538


C:\Users\Helia\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Helia\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

In [14]:
def output_prediction(testdata, ypred):
    testdata["score"] = ypred.tolist()
    df = pd.DataFrame({"srch_id": testdata["srch_id"].values,
                  "prop_id": testdata["prop_id"]})
    return df

In [15]:
gnbpred = output_prediction(testdata, gnb)
gnbpred

,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
...,...,...
4995,321,75719
4996,321,77067
4997,321,81826
4998,321,98611


In [16]:
dtpred = output_prediction(testdata, dt)
dtpred

,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
...,...,...
4995,321,75719
4996,321,77067
4997,321,81826
4998,321,98611


In [17]:
knnpred = output_prediction(testdata, knn)
knnpred

,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
...,...,...
4995,321,75719
4996,321,77067
4997,321,81826
4998,321,98611


In [18]:
sgdpred = output_prediction(testdata, sgd)
sgdpred

,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
...,...,...
4995,321,75719
4996,321,77067
4997,321,81826
4998,321,98611


In [19]:
lrpred = output_prediction(testdata, lr)
lrpred

,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
...,...,...
4995,321,75719
4996,321,77067
4997,321,81826
4998,321,98611
